In [2]:
import csv
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import numpy as np 


### Task 1

In [3]:
f = open('ManUtd_task1.csv', 'w', encoding="utf-8",  newline='')
writer = csv.writer(f)


In [4]:
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:ManUtd').get_items()):
    if i>1000:
        break
    writer.writerow([tweet.rawContent])

f.close()

### Task 2

In [64]:
df = pd.read_csv('ManUtd_task1.csv', header = None, sep=',')
df.head(10)

,0
0,"🏆 See where the young Reds will be travelling,..."
1,Sending good vibes to @ManUtdWomen ahead of ou...
2,A first United Player of the Month award for @...
3,Eight iconic #WorldCup moments.\nOne winner.\n...
4,Our newest fan channel is live 👀\n\nJoin our @...
5,A record to be proud of 🧬\n\n#MUFC || #MUAcademy
6,ℹ️ Club statement.\n\n#MUFC
7,Charge up over the winter break 🔋\n\n#MUFC
8,4️⃣ appearances\n2️⃣ goals\n2️⃣ assists\n\nWel...
9,🌟 A starring role in November...\n\nYou've vot...


In [65]:
def byebyeEmoji(text):
    regrex_pattern = re.compile(pattern = "["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d" 
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u2139" #remove block i character 
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [66]:
file=open('stop_words.txt','r')
stopWords = file.read()
punct =  r'[^ \\t\d\w\.]'
spaces = r"\s{2,}" 
digits = r'[0-9]'
links = r'http\S+'
underscore = r'_'

df[0]=df[0].str.lower()
df = df[0].apply(byebyeEmoji)
df = df.replace(to_replace =links, value = '', regex = True)
df = df.replace(to_replace =underscore, value = '', regex = True)
df = df.replace(to_replace =digits, value = '', regex = True)
df = df.replace(to_replace =stopWords, value = '', regex = True)
df = df.replace(to_replace ="@[A-Za-z0-9_]+", value = ' ', regex = True)
df = df.replace(to_replace ="#[A-Za-z0-9_]+", value = ' ', regex = True)
df = df.replace(to_replace ="www.\S+", value = ' ', regex = True)
df = df.replace(to_replace =punct, value = '', regex = True)
df = df.replace(to_replace =r'\.', value = '', regex = True)
df = df.replace(to_replace =spaces, value = ' ', regex = True)



In [68]:
df.head()

0     see where the young reds will be travelling a...
1    sending good vibes to ahead of our big clash w...
2        a first united player of the month award for 
3    eight iconic momentsone winnerfeaturing haircu...
4    our newest fan channel is live join our for si...
Name: 0, dtype: object

In [44]:
df.to_csv('ManUtd_task2.csv', index=False, header = False)

### Task 3

In [45]:
df = pd.read_csv('ManUtd_task2.csv', header=None)
df.head()

,0
0,see where the young reds will be travelling a...
1,sending good vibes to ahead of our big clash w...
2,a first united player of the month award for
3,eight iconic momentsone winnerfeaturing haircu...
4,our newest fan channel is live join our for si...


In [47]:
train, test = train_test_split( df[0], test_size = 0.2, shuffle = True)

In [48]:
print(df.shape)
print(train.shape)
print(test.shape)

(963, 1)
(770,)
(193,)


In [49]:
def create_vocabulary(data): 
    v = []
    for  i in data: 
        i = str(i)
        splitRow = i.split()
        # print(splitRow)
        vocab = [j for j in splitRow]
        v.append(vocab)
    flat_v = [item for sublist in v for item in sublist]
    vocab = np.array(flat_v)
    vocabFinal = np.unique(vocab)
    # print(vocabFinal)
    return vocabFinal
    

In [69]:
print(len(create_vocabulary(train)))

1532


In [50]:
def create_bag_of_words(testData, trainData):
    vocab = create_vocabulary(trainData)
    bag_of_word = []
    for i in testData: 
        i = str(i)
        splitRow = i.split()
        temp_vocab = np.zeros(len(vocab))
        for row in splitRow:
            vocab_found = True
            for j in range (len(vocab)):
                if row == vocab[j]:
                    vocab_found = False
                    temp_vocab[j] += 1
            if vocab_found == True:
                    temp_vocab= temp_vocab + 1
        bag_of_word.append(temp_vocab)
    return bag_of_word
                
        

In [51]:
a = create_bag_of_words(train, train)

for i in range(10):
    print(a[i])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [52]:
a = create_bag_of_words(test, train)
for i in range(10):
    print(a[i])

[3. 3. 3. ... 3. 3. 3.]
[2. 2. 2. ... 2. 2. 2.]
[3. 3. 3. ... 3. 3. 3.]
[4. 4. 4. ... 4. 4. 4.]
[1. 1. 1. ... 1. 1. 1.]
[2. 2. 2. ... 2. 2. 2.]
[0. 0. 1. ... 0. 0. 0.]
[4. 4. 4. ... 4. 4. 4.]
[2. 2. 2. ... 2. 2. 2.]
[1. 1. 1. ... 1. 1. 1.]
